In [1]:
import sys
from pathlib import Path
from threading import Thread
from subprocess import Popen, PIPE
from collections import defaultdict

import numpy as np
import pandas as pd

In [2]:
!ls ../out

2022-08-14T22:24:50.595949_6397.73_sa_5_0.5
2022-08-14T22:27:21.957662_6670.33_sa_double_tl
2022-08-14T22:41:23.433876_7152.92_sa_5x_tl
2022-08-15T07:28:46.169766_6590.63_radius3
2022-08-15T11:57:10.287165_7293.30_5x_tl
2022-08-15T12:12:25.968346_4774.42_5x_tl_debug
2022-08-15T20:06:16.422917_6690.59_erase2
2022-08-15T20:17:02.557507_6757.84_gcc
2022-08-16T00:24:18.000224_6831.50_attraction
2022-08-16T02:04:03.826319_6613.95_juggernaut
2022-08-16T02:30:02.892914_6846.77_attract_near
2022-08-16T03:57:54.818855_6960.94_sa_10
2022-08-16T11:28:11.134584_7272.17_tuned


In [3]:
scores = {}

def read_stream(name, in_file, out_file):
    for line in in_file:
        #print(f"[{name}] {line.strip()}", file=out_file)
        try:
            scores[name] = int(line.strip().split()[-1])
        except:
            pass

def run(cmd, name):
    proc = Popen(cmd, stdout=PIPE, stderr=PIPE, universal_newlines=True, shell=True)
    stdout_thread = Thread(target=read_stream, args=(name, proc.stdout, sys.stdout))
    stderr_thread = Thread(target=read_stream, args=(name, proc.stderr, sys.stderr))
    stdout_thread.start()
    stderr_thread.start()
    proc.wait()
    return proc

N = 100
for out_dir in sorted(Path("../out").iterdir()):
    for i in range(N):
        out_file = out_dir / f"{i:04d}.txt"
        cmd = f"../tools/target/release/vis ../tools/in/{i:04d}.txt {out_file}"
        name = out_dir.name[out_dir.name.index("_") + 9:]
        run(cmd, (name, i))

In [4]:
dict_out_dir_to_scores = defaultdict(lambda: [0] * N)
for (out_dir, i), score in scores.items():
    dict_out_dir_to_scores[out_dir][i] = score

In [5]:
columns = ["radius3", "erase2", "gcc", "attraction", "attract_near", "sa_10", "tuned"]
df = pd.DataFrame(dict_out_dir_to_scores)[columns]
df.corr()

,radius3,erase2,gcc,attraction,attract_near,sa_10,tuned
radius3,1.000000,0.845305,0.828297,0.826099,0.798595,0.767785,0.799191
erase2,0.845305,1.000000,0.931954,0.872396,0.824882,0.856498,0.806428
gcc,0.828297,0.931954,1.000000,0.873411,0.839438,0.864427,0.841084
attraction,0.826099,0.872396,0.873411,1.000000,0.850996,0.824446,0.832827
attract_near,0.798595,0.824882,0.839438,0.850996,1.000000,0.794619,0.817579
sa_10,0.767785,0.856498,0.864427,0.824446,0.794619,1.000000,0.825307
tuned,0.799191,0.806428,0.841084,0.832827,0.817579,0.825307,1.000000


In [6]:
df.values.argmax(1)

array([3, 1, 3, 6, 4, 6, 6, 6, 6, 6, 5, 6, 3, 6, 6, 6, 6, 3, 0, 5, 5, 4,
       1, 6, 6, 4, 5, 6, 6, 6, 6, 5, 6, 2, 6, 4, 3, 6, 6, 6, 6, 2, 1, 3,
       6, 6, 0, 5, 6, 0, 4, 6, 2, 6, 0, 6, 2, 4, 3, 4, 6, 6, 6, 6, 5, 5,
       1, 6, 1, 6, 6, 5, 6, 5, 3, 5, 5, 1, 4, 0, 2, 4, 6, 4, 3, 5, 5, 6,
       4, 6, 4, 6, 6, 6, 6, 0, 6, 4, 3, 6])

In [7]:
np.bincount(df.values.argmax(1))

array([ 6,  6,  5, 10, 13, 14, 46])

In [8]:
ns = []
ks = []
for i in range(N):
    cmd = f"head -n 1 ../tools/in/{i:04d}.txt"
    nk = !{cmd}
    n, k = map(int, nk[0].split())
    ns.append(n)
    ks.append(k)
df["n"] = ns
df["k"] = ks

In [9]:
df.mean()

radius3         6590.63
erase2          6690.59
gcc             6757.84
attraction      6831.50
attract_near    6846.77
sa_10           6960.94
tuned           7272.17
n                 30.45
k                  3.50
dtype: float64

In [10]:
df.groupby("k").mean()[columns].style.background_gradient(cmap="YlOrRd", axis=1)

,radius3,erase2,gcc,attraction,attract_near,sa_10,tuned
k,,,,,,,
2,6255.800000,6273.520000,6472.840000,6462.360000,6367.200000,6453.640000,6677.360000
3,6841.880000,7076.200000,7072.080000,7096.360000,7176.960000,6958.360000,7391.520000
4,6775.360000,6952.880000,6898.600000,7028.080000,6988.720000,7247.280000,7358.440000
5,6489.480000,6459.760000,6587.840000,6739.200000,6854.200000,7184.480000,7661.360000


In [11]:
df["knn"] = 100 * df["k"] / (df["n"] * df["n"])
df["knn_bin"] = pd.qcut(df["knn"], 5)
df.groupby("knn_bin").mean()[columns].style.background_gradient(cmap="YlOrRd", axis=1)

,radius3,erase2,gcc,attraction,attract_near,sa_10,tuned
knn_bin,,,,,,,
"(0.13, 0.238]",6924.150000,7050.000000,7077.600000,6965.200000,7046.900000,7111.900000,7137.600000
"(0.238, 0.322]",7162.150000,7546.050000,7611.600000,7592.100000,7511.500000,7705.850000,8326.500000
"(0.322, 0.448]",7024.550000,7187.950000,7281.050000,7561.900000,7463.200000,7677.550000,8065.900000
"(0.448, 0.62]",6900.809524,6810.095238,6889.428571,7088.857143,6995.047619,7042.428571,7387.476190
"(0.62, 0.926]",4838.368421,4756.157895,4826.368421,4836.842105,5123.631579,5173.526316,5341.052632
